In [ ]:
import enoslib as en
from grid5000 import Grid5000
from pathlib import Path

conf_file = Path.home() / ".python-grid5000.yaml"
gk = Grid5000.from_yaml(conf_file)
gk.sites.list() # if credentials were written adequatly in .env, this should work

Reserve machines

In [ ]:
import enoslib as en
from grid5000 import Grid5000
from pathlib import Path

conf_file = Path.home() / ".python-grid5000.yaml"
gk = Grid5000.from_yaml(conf_file)
gk.sites.list() # if credentials were written adequatly in .env, this should work

job_name = 'dev'

conf = (
    en.G5kConf.from_settings(
        job_name=job_name,
        job_type=[],#"deploy"],
        # key="/home/adrien/.ssh/grid",
        #env_name="ubuntu2204-min",
        walltime="08:00:00"
    )    
    .add_machine(roles=["client"], cluster="grimoire", nodes=1)
    .add_machine(roles=["server"], cluster="grimoire", nodes=1)
    # .add_machine(roles=["client"], cluster="grimoire", nodes=1)
    # .add_machine(roles=["client"], servers=["uvb-41.sophia.grid5000.fr"]) #cluster="grimoire", nodes=1)
    # .add_machine(roles=["server"], servers=["uvb-43.sophia.grid5000.fr"]) #cluster="grimoire", nodes=1)
    # .add_machine(roles=["client"], servers=["grimoire-3.nancy.grid5000.fr"]) #cluster="grimoire", nodes=1)
    # .add_machine(roles=["server"], servers=["grimoire-4.nancy.grid5000.fr"]) #cluster="grimoire", nodes=1)
)

provider = en.G5k(conf)

roles, networks = provider.init()

In [ ]:
results = en.run_command("sudo-g5k apt update", roles=roles["client"]+roles["server"])
results = en.run_command("sudo-g5k apt install -y apt-transport-https ca-certificates curl software-properties-common", roles=roles["client"]+roles["server"])
results = en.run_command("curl -fsSL https://download.docker.com/linux/debian/gpg | sudo-g5k gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg", roles=roles["client"]+roles["server"])
results = en.run_command('echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/debian $(lsb_release -cs) stable" | sudo-g5k tee /etc/apt/sources.list.d/docker.list > /dev/null', roles=roles["client"]+roles["server"])
results = en.run_command("sudo-g5k apt update", roles=roles["client"]+roles["server"])
results = en.run_command("apt-cache policy docker-ce", roles=roles["client"]+roles["server"])
results = en.run_command("sudo-g5k apt install -y docker-ce", roles=roles["client"]+roles["server"])


In [ ]:
results = en.run_command("sudo-g5k apt install apt-transport-https curl gnupg -y", roles=roles["client"]+roles["server"])
results = en.run_command("curl -fsSL https://bazel.build/bazel-release.pub.gpg | gpg --dearmor >bazel-archive-keyring.gpg", roles=roles["client"]+roles["server"])
results = en.run_command("sudo-g5k mv bazel-archive-keyring.gpg /usr/share/keyrings", roles=roles["client"]+roles["server"])
results = en.run_command('echo "deb [arch=amd64 signed-by=/usr/share/keyrings/bazel-archive-keyring.gpg] https://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo-g5k tee /etc/apt/sources.list.d/bazel.list', roles=roles["client"]+roles["server"])
results = en.run_command("sudo-g5k apt update && sudo-g5k apt install -y bazel", roles=roles["client"]+roles["server"])
#results = en.run_command("sudo-g5k apt update && sudo-g5k apt full-upgrade -y", roles=roles["client"])

In [ ]:
# results = en.run_command("scp -r rdma-vm:./g5k/envoy-docker-build.tar.gz /tmp", roles=roles["client"]+roles["server"])
results = en.run_command("cp /home/mdeneuville/envoy-docker-build.tar.gz /tmp", roles=roles["client"]+roles["server"])
results = en.run_command("cd /tmp && sudo-g5k tar -xzvf envoy-docker-build.tar.gz", roles=roles["client"]+roles["server"])
results = en.run_command("cd /tmp && sudo chown -R root:docker envoy-docker-build && sudo chmod 755 -R envoy-docker-build", roles=roles["client"]+roles["server"])
results = en.run_command("cd /home/mdeneuville/envoy && ./ci/run_envoy_docker.sh './ci/do_ci.sh bazel.release.server_only'", roles=roles["client"])
results = en.run_command("cd /home/mdeneuville/envoy && ./ci/run_envoy_docker.sh './ci/do_ci.sh bazel.release.server_only'", roles=roles["server"])

Setup experiments

In [ ]:
results = en.run_command("sudo-g5k apt install -y memcached", roles=roles["server"])

In [ ]:
results = en.run_command("ip addr show ib0 | grep inet | awk '{ print $2 }' | cut -d/ -f1 | head -n 1", roles=roles["client"])
client_ip = results[0].payload['stdout']
print(client_ip)

results = en.run_command("ip addr show ib0 | grep inet | awk '{ print $2 }' | cut -d/ -f1 | head -n 1", roles=roles["server"])
server_ip = results[0].payload['stdout']
print(server_ip)

sender_type = ['sender', 'sender_rdma', 'sender_rdma_write_multip_write', 'sender_rdma_write_multip_read', 'sender_rdma_write_nomultip_read']
SENDER_TYPE = ['Sender', 'SenderRDMA', 'SenderRDMAWriteMultipWrite', 'SenderRDMAWriteMultipRead', 'SenderRDMAWriteNomultipRead']

receiver_type = ['receiver', 'receiver_rdma', 'receiver_rdma_write_multip_write', 'receiver_rdma_write_multip_read', 'receiver_rdma_write_nomultip_read']
RECEIVER_TYPE = ['Receiver', 'ReceiverRDMA', 'ReceiverRDMAWriteMultipWrite', 'ReceiverRDMAWriteMultipRead', 'ReceiverRDMAWriteNomultipRead']

In [ ]:
# memtier params
payload_sizes = ['10-100', '100-1000', '1000-10000']
simult_req = ['1', '10', '100']
# RDMA filters params
# payload_bounds = ['100', '300', '1000', '3000', '6000']
# circle_sizes = ['100', '200', '500']
payload_bounds = ['1500']
circle_sizes = ['200']
test_time = 90 # in seconds

# RDMA filters dictionary
benchmarks_custom = {}
for size in payload_sizes:
    benchmarks_custom[size] = {}
    for n in simult_req:
        benchmarks_custom[size][n] = {}
        for payload_bound in payload_bounds:
            benchmarks_custom[size][n][payload_bound] = {}
            for circle_size in circle_sizes:
                benchmarks_custom[size][n][payload_bound][circle_size] = {}
                for i in range(len(sender_type)):
                    benchmarks_custom[size][n][payload_bound][circle_size][i] = f"./memtier_benchmark -s 127.0.0.1 -p 3001 -P memcache_text -c 1 -t 1 --test-time={test_time} --data-size-range={size} --pipeline={n} --print-percentiles 50,99,99.9, 99.99, 99.999 --hdr-file-prefix=latencies_{sender_type[i]}_{size}_{n}_{payload_bound}_{circle_size} --json-out-file=json_{sender_type[i]}_{size}_{n}_{payload_bound}_{circle_size}"

# tcp_proxy dictionary
benchmarks_tcpproxy = {}
for size in payload_sizes:
    benchmarks_tcpproxy[size] = {}
    for n in simult_req:
        benchmarks_tcpproxy[size][n] = f"./memtier_benchmark -s 127.0.0.1 -p 3001 -P memcache_text -c 1 -t 1 --test-time={test_time} --data-size-range={size} --pipeline={n} --print-percentiles 50,99,99.9, 99.99, 99.999 --hdr-file-prefix=latencies_tcpproxy_{size}_{n} --json-out-file=json_tcpproxy_{size}_{n}"

In [ ]:

import subprocess
import time

i = 0
for size in payload_sizes:
    for n in simult_req:
        for payload_bound in payload_bounds:
            for circle_size in circle_sizes:
                for i in range(len(sender_type)):
                    if (i > -1):
                        # Custom filters
                        results = en.run_command(f"""export CLIENT_IP={client_ip} && export SERVER_IP={server_ip} && \
                            export sender_type={sender_type[i]} && export SENDER_TYPE={SENDER_TYPE[i]} && \
                            export receiver_type={receiver_type[i]} && export RECEIVER_TYPE={RECEIVER_TYPE[i]} && \
                            export payload_bound={payload_bound} && \
                            export circle_size={circle_size} && \
                            cd /home/mdeneuville/envoy && envsubst < config_template_files/template_client.yaml > config_files/client.yaml""", roles=roles["client"])

                        results = en.run_command(f"""export CLIENT_IP={client_ip} && export SERVER_IP={server_ip} && \
                            export sender_type={sender_type[i]} && export SENDER_TYPE={SENDER_TYPE[i]} && \
                            export receiver_type={receiver_type[i]} && export RECEIVER_TYPE={RECEIVER_TYPE[i]} && \
                            export payload_bound={payload_bound} && \
                            export circle_size={circle_size} && \
                            cd /home/mdeneuville/envoy && envsubst < config_template_files/template_server.yaml > config_files/server.yaml""", roles=roles["server"])

                        # Launch Envoy proxy
                        results = en.run_command("cd /home/mdeneuville/envoy && /tmp/envoy-docker-build/envoy/source/exe/envoy/envoy -c config_files/client.yaml", background=True, roles=roles["client"])
                        results = en.run_command("cd /home/mdeneuville/envoy && /tmp/envoy-docker-build/envoy/source/exe/envoy/envoy -c config_files/server.yaml", background=True, roles=roles["server"])

                        # Launch benchmark
                        results = en.run_command(f'cd /home/mdeneuville/memtier_benchmark  && {benchmarks_custom[size][n][payload_bound][circle_size][i]}', roles=roles["client"])
                        for res in results:
                            print(sender_type[i])
                            print(res.payload['stdout'])
                            print("")
                        
                        # Kill proxies
                        try:
                            results = en.run_command('pgrep -f "/tmp/envoy-docker-build/envoy/source/exe/envoy/envoy" | xargs kill', roles=roles["client"]+roles["server"])
                        except:
                            print("Failed successfully")
                        
                        # Reset memcached
                        results = en.run_command('echo "flush_all" | nc -q 2 localhost 11211', roles=roles["server"])

                        # Sleep for 5 seconds
                        time.sleep(5)
                        i += 1

        # TCP proxy
        results = en.run_command(f"""export CLIENT_IP={client_ip} && export SERVER_IP={server_ip} && \
            cd /home/mdeneuville/envoy && envsubst < config_template_files/template_client_tcp_proxy.yaml > config_files/client_tcp_proxy.yaml""", roles=roles["client"])

        results = en.run_command(f"""export CLIENT_IP={client_ip} && export SERVER_IP={server_ip} && \
            cd /home/mdeneuville/envoy && envsubst < config_template_files/template_server_tcp_proxy.yaml > config_files/server_tcp_proxy.yaml""", roles=roles["server"])

        # Launch Envoy proxy
        results = en.run_command("cd /home/mdeneuville/envoy && /tmp/envoy-docker-build/envoy/source/exe/envoy/envoy -c config_files/client_tcp_proxy.yaml", background=True, roles=roles["client"])
        results = en.run_command("cd /home/mdeneuville/envoy && /tmp/envoy-docker-build/envoy/source/exe/envoy/envoy -c config_files/server_tcp_proxy.yaml", background=True, roles=roles["server"])

        # Launch benchmark
        results = en.run_command(f"cd /home/mdeneuville/memtier_benchmark && {benchmarks_tcpproxy[size][n]}", roles=roles["client"])
        for res in results:
            print("TCP_PROXY")
            print(res.payload['stdout'])
            print("")

        # Kill proxies
        try:
            results = en.run_command('pgrep -f "/tmp/envoy-docker-build/envoy/source/exe/envoy/envoy" | xargs kill', roles=roles["client"]+roles["server"])
        except:
            print("Failed successfully")

        # Reset memcached
        results = en.run_command('echo "flush_all" | nc -q 2 localhost 11211', roles=roles["server"])

        # Sleep for 5 seconds
        time.sleep(5)

# Copy results locally
!scp nancy.g5k:/home/mdeneuville/memtier_benchmark/json* results/
!scp nancy.g5k:/home/mdeneuville/memtier_benchmark/latencies* results/

In [ ]:
results = en.run_command("cd /home/mdeneuville/envoy && /tmp/envoy-docker-build/envoy/source/exe/envoy/envoy -c config_files/client.yaml", background=True, roles=roles["client"])
results = en.run_command("cd /home/mdeneuville/envoy && /tmp/envoy-docker-build/envoy/source/exe/envoy/envoy -c config_files/server.yaml", background=True, roles=roles["server"])

In [ ]:
results = en.run_command("cd /home/mdeneuville/memtier_benchmark && ./memtier_benchmark --test-time=5 -s localhost -p 3001 -P memcache_text -c 1 -t 1 --hdr-file-prefix=latencies --json-out-file=res_json", roles=roles["client"])
for res in results:
    print(res.payload['stdout'])

!scp sophia.g5k:/home/mdeneuville/memtier_benchmark/res_json results/
sender_type = "sender_type1"
name = f'{sender_type}'
!mv results/res_json results/res_json_{name}
!scp sophia.g5k:/home/mdeneuville/memtier_benchmark/latencies* results/
import subprocess
directory = "results"
command = f'for file in {directory}/latencies*; do mv "$file" "{directory}/{name}_${{file##*/}}"; done'
subprocess.run(["bash", "-c", command], check=True)


In [ ]:
try:
    results = en.run_command('pgrep -f "/tmp/envoy-docker-build/envoy/source/exe/envoy/envoy" | xargs kill', roles=roles["client"]+roles["server"])
except:
    print("Failed successfully")

In [ ]:
# !rm -rf results/*

In [ ]:
provider.destroy()